In [35]:
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets

import numpy as np


In [36]:
time1 = "051105 061512"
ra, dec = 10., -40.

In [37]:
version = find_response_version(time1)
rsp_base = ResponseDataRMF.from_version(version)
dets = get_live_dets(time=time1, event_types=["single"])

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [38]:
ein = np.array(np.linspace(20,1000,4))
ebounds = np.array(np.linspace(20,1000,4))
d = 0

In [39]:
rsp1 = ResponseRMFGenerator.from_time(time1,  d,
                                            ebounds, ein,
                                            rsp_base)
sd1 = SPIDRM(rsp1, ra, dec)
print(ein)
print(sd1.matrix)

[  20.          346.66666667  673.33333333 1000.        ]
[[13.823791    2.70435285  1.79275684]
 [ 0.          8.53337789  1.70787271]
 [ 0.          0.          6.04052616]]


In [40]:
ein = np.array(np.linspace(20,1000,7))
ebounds = np.array(np.linspace(20,1000,4))


In [41]:
rsp2 = ResponseRMFGenerator.from_time(time1,  d,
                                            ebounds, ein,
                                            rsp_base)
sd2 = SPIDRM(rsp2, ra, dec)
print(ein)
print(sd2.matrix)

[  20.          183.33333333  346.66666667  510.          673.33333333
  836.66666667 1000.        ]
[[14.06299834 15.31597976  2.85573938  2.29760791  1.91975918  1.63880981]
 [ 0.          0.          8.38959361  8.25712902  1.8686086   1.15987021]
 [ 0.          0.          0.          0.          5.73556994  5.82934172]]


In [42]:
print(sd1.matrix[0,0])
print(sd2.matrix[0,:2])

13.82379100056793
[14.06299834 15.31597976]


In [43]:
# should the value from sd1 not lie inbetween the two sd2 values?

In [3]:
import astropy.io.fits as fits
from astropy.table import Table
import astropy.time as at
from datetime import datetime
import numpy as np


In [4]:
data_path = "./main_files/spimodfit_comparison_sim_source/pyspi_real_bkg/0374"

# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])

In [5]:
t

PTID_ISOC,PTID_SPI,OBT_START,OBT_END,TSTART,TSTOP,TELAPSE,RA_SPIX,DEC_SPIX,RA_SPIZ,DEC_SPIZ
,,,,d,d,s,deg,deg,deg,deg
str8,str15,uint16[4],uint16[4],float64,float64,float64,float32,float32,float32,float32
03740001,03740001.000000,0 .. 63737,0 .. 43300,2135.2520391666667,2135.2588705421003,590.230837467243,14.359367,-46.588104,229.94302,-37.57425
03740002,03740002.000000,0 .. 63308,0 .. 46651,2135.260557685185,2135.291722883317,2692.673118598759,13.767507,-46.369335,230.1961,-37.48927
03740003,03740003.000000,0 .. 61659,0 .. 57115,2135.293150277778,2135.3240407251647,2668.9346542203566,11.059443,-45.68437,230.99942,-36.81915
03740004,03740004.000000,0 .. 60025,0 .. 15008,2135.3254650925924,2135.3563160307904,2665.521060307219,8.418544,-44.942196,231.72835,-36.096745
03740005,03740005.000000,0 .. 58374,0 .. 31713,2135.358057685185,2135.3885297749794,2632.788558230095,5.848483,-44.144936,232.3689,-35.334618
03740006,03740006.000000,0 .. 56748,0 .. 32146,2135.389909537037,2135.420778061946,2667.0405521188513,3.3409438,-43.29776,232.92168,-34.53164
03740007,03740007.000000,0 .. 55089,0 .. 44763,2135.422131759259,2135.4530923111793,2674.9916858956567,4.515532,-41.49371,234.48235,-36.024696
03740008,03740008.000000,0 .. 53411,0 .. 51792,2135.454446574074,2135.4853493518517,2669.9999999807915,6.964257,-42.318604,233.91713,-36.858116


In [2]:
import numpy as np

a = np.geomspace(18,2000,100)
print(a)

[  18.           18.87716284   19.79707094   20.76180732   21.77355654
   22.83460958   23.94736907   25.11435476   26.33820913   27.62170348
   28.96774414   30.37937907   31.85980476   33.41237347   35.04060082
   36.74817376   38.5389589    40.41701127   42.38658351   44.45213551
   46.61834447   48.89011555   51.27259292   53.77117143   56.39150884
   59.13953862   62.02148337   65.04386896   68.21353922   71.53767154
   75.02379305   78.67979768   82.51396405   86.53497422   90.75193331
   95.17439018   99.81235897  104.67634187  109.77735283  115.12694253
  120.73722452  126.62090267  132.79129992  139.26238846  146.04882135
  153.16596573  160.62993759  168.45763826  176.66679273  185.2759897
  194.30472377  203.7734395   213.70357777  224.11762428  235.03916048
  246.49291701  258.50482962  271.10209797  284.3132472   298.16819249
  312.69830684  327.9364921   343.91725347  360.67677762  378.25301464
  396.68576399  416.01676461  436.28978942  457.55074445  479.84777279
  503.2

In [4]:
print(a[:70])

[ 18.          18.87716284  19.79707094  20.76180732  21.77355654
  22.83460958  23.94736907  25.11435476  26.33820913  27.62170348
  28.96774414  30.37937907  31.85980476  33.41237347  35.04060082
  36.74817376  38.5389589   40.41701127  42.38658351  44.45213551
  46.61834447  48.89011555  51.27259292  53.77117143  56.39150884
  59.13953862  62.02148337  65.04386896  68.21353922  71.53767154
  75.02379305  78.67979768  82.51396405  86.53497422  90.75193331
  95.17439018  99.81235897 104.67634187 109.77735283 115.12694253
 120.73722452 126.62090267 132.79129992 139.26238846 146.04882135
 153.16596573 160.62993759 168.45763826 176.66679273 185.2759897
 194.30472377 203.7734395  213.70357777 224.11762428 235.03916048
 246.49291701 258.50482962 271.10209797 284.3132472  298.16819249
 312.69830684 327.9364921  343.91725347 360.67677762 378.25301464
 396.68576399 416.01676461 436.28978942 457.55074445 479.84777279]


In [5]:
b = np.geomspace(18, 3000, 75)
print(b)

[  18.           19.28845704   20.66914306   22.1486599    23.73408195
   25.43299001   27.25350751   29.20433939   31.29481366   33.53492605
   35.93538782   38.5076769    41.26409288   44.21781572   47.38296884
   50.77468662   54.40918677   58.30384788   62.47729252   66.94947628
   71.74178319   76.87712797   82.38006558   88.27690869   94.5958534
  101.36711416  108.62306818  116.39841026  124.73031869  133.65863301
  143.22604453  153.47830042  164.4644225   176.2369416   188.85214876
  202.3703644   216.85622672  232.37900077  249.01290969  266.83748957
  285.93796975  306.40568039  328.33848913  351.8412691   377.02640031
  404.01430706  432.93403374  463.92386183  497.13197116  532.71714841
  570.84954634  611.71149742  655.49838565  702.41958081  752.69943954
  806.57837818  864.31402226  926.18243842  992.4794544  1063.52207355
 1139.64999066 1221.22721616 1308.64381671 1402.3177803  1502.69701491
 1610.26149019 1725.52553247 1849.04028403 1981.39633848 2123.22656463
 2275.2

In [8]:
print(b[:50])

[ 18.          19.28845704  20.66914306  22.1486599   23.73408195
  25.43299001  27.25350751  29.20433939  31.29481366  33.53492605
  35.93538782  38.5076769   41.26409288  44.21781572  47.38296884
  50.77468662  54.40918677  58.30384788  62.47729252  66.94947628
  71.74178319  76.87712797  82.38006558  88.27690869  94.5958534
 101.36711416 108.62306818 116.39841026 124.73031869 133.65863301
 143.22604453 153.47830042 164.4644225  176.2369416  188.85214876
 202.3703644  216.85622672 232.37900077 249.01290969 266.83748957
 285.93796975 306.40568039 328.33848913 351.8412691  377.02640031
 404.01430706 432.93403374 463.92386183 497.13197116 532.71714841]


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('./main_files'))

from PointingClusters import *



/home/moej56153/.pyenv/versions/3.9.13/envs/MT2/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


Using the irfs that are valid between Start and 03/07/06 06:00:00 (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 03/07/06 06:00:00 and 04/07/17 08:20:06 (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 09/02/19 09:59:57 and 10/05/27 12:45:00 (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)


In [9]:
data_folder = "./main_files/SPI_data/0043"

outliers = load_outliers("./main_files/SPI_data/low_energy_outliers.pickle","0043")
pointings = load_clusters(data_folder)

In [10]:
pointings, outliers

(((('004300060010', './main_files/SPI_data/0043'),
   ('004300080010', './main_files/SPI_data/0043')),
  (('004300070010', './main_files/SPI_data/0043'),
   ('004300090010', './main_files/SPI_data/0043')),
  (('004300100010', './main_files/SPI_data/0043'),
   ('004300130010', './main_files/SPI_data/0043')),
  (('004300110010', './main_files/SPI_data/0043'),
   ('004300140010', './main_files/SPI_data/0043')),
  (('004300150010', './main_files/SPI_data/0043'),
   ('004300170010', './main_files/SPI_data/0043')),
  (('004300160010', './main_files/SPI_data/0043'),
   ('004300180010', './main_files/SPI_data/0043')),
  (('004300200010', './main_files/SPI_data/0043'),
   ('004300220010', './main_files/SPI_data/0043')),
  (('004300210010', './main_files/SPI_data/0043'),
   ('004300230010', './main_files/SPI_data/0043')),
  (('004300240010', './main_files/SPI_data/0043'),
   ('004300270010', './main_files/SPI_data/0043')),
  (('004300250010', './main_files/SPI_data/0043'),
   ('004300280010', '.

In [11]:
remove_outlier_clusters(pointings, outliers)

AssertionError: Outliers do not correspond to Clusters